### Парсинг сделан по ноутбуку: Итоговое задание Альмиры, Андрея Скороходова и Соколова Александра  
### по Проекту 6. Car Price prediction 

### 1. Импорт библиотек

In [222]:
import pandas as pd
import requests, json
import time
import os

import os

pd.options.mode.chained_assignment = None

# этот блок закомментирован так как используется только на kaggle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = '/kaggle/input/sf-dst-car-price/'
PATH_to_file_data = '/kaggle/input/all-brands-and-models-for-auto-ru-09-09-2020/'

# # # этот блок закомментирован так как используется только локальной машине
# from importlib import reload
# print(os.listdir('./data'))
#PATH_to_file = './data/'

### 2. Импорт данных о марках и моделях

In [223]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


### 2. Отбор марок для парсинга

In [224]:
# кол-во объявлений на сайте на 9.9.2020 
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [225]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)

In [226]:
# посмотрим на список получившихся моделей
print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GEELY,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LADA (ВАЗ),LAND ROVER,LEXUS,MAZDA,MERCEDES-BENZ,MINI,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO,ГАЗ,УАЗ


In [227]:
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

In [228]:
# проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300:= {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300:= 36
список марок, которые нужно проверить := GREAT WALL LAND ROVER MERCEDES-BENZ SSANGYONG


In [229]:
# эта строка позволяет проверить значение с пробелом или с дефисом для корректировки
dict_of_num_cars_of_models['GREAT WALL']

420

In [230]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [231]:
# еще раз проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300 := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300 := 36
список марок, которые нужно проверить :=


In [232]:
dict_of_num_cars_of_models

{'AUDI': 4093,
 'BMW': 7423,
 'CADILLAC': 321,
 'CHERY': 593,
 'CHEVROLET': 3670,
 'CHRYSLER': 316,
 'CITROEN': 1142,
 'DAEWOO': 1024,
 'DODGE': 413,
 'FORD': 4812,
 'GEELY': 302,
 'HONDA': 1179,
 'HYUNDAI': 5712,
 'INFINITI': 1041,
 'JAGUAR': 308,
 'JEEP': 466,
 'KIA': 5595,
 'LEXUS': 985,
 'MAZDA': 2311,
 'MINI': 313,
 'MITSUBISHI': 3935,
 'NISSAN': 5776,
 'OPEL': 3131,
 'PEUGEOT': 1636,
 'PORSCHE': 714,
 'RENAULT': 2771,
 'SKODA': 2810,
 'SUBARU': 934,
 'SUZUKI': 878,
 'TOYOTA': 4648,
 'VOLKSWAGEN': 7109,
 'VOLVO': 2069,
 'GREAT_WALL': 420,
 'LAND_ROVER': 1616,
 'MERCEDES': 8005,
 'SSANG_YONG': 718}

In [233]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())
#list_of_relevant_marks = list({'AUDI': 4093, 'BMW': 7423})

In [234]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 89189


### 3. Подготовка к парсингу

In [243]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=3e2b10b5c34f56ca20cceb9967b1ba5adae281a7cee3f71a; autoru_sid=a%3Ag5fc1088e2huo0e0707tbqbf6hdt87ue.1cf42524c68db40db0402c2f364777a6%7C1606486158891.604800.LhLPEOozUY03iF5B3kn3FA.qy-ydMZY2czBIGlI3fKFSKcu73RyES_nYoVcGNl0v6U; autoruuid=g5fc1088e2huo0e0707tbqbf6hdt87ue.1cf42524c68db40db0402c2f364777a6; suid=a729160f4e55a4515fe61cf90f024d84.23fd0a56a5dc1f8102fd05613b599d99; from_lifetime=1606833021149; from=direct; yuidcs=1; yuidlt=1; yandexuid=668098521593180782; my=YwA%3D; crookie=HdxerBu/82nso0BNFH4SouFhOnkRgJyRSiC2/R8ZkXxDNh8ymT37QTWipJdlTrIX9KgMumqPZ6BSqUGMFPf+JuDD3UE=; cmtchd=MTYwNjQ4NjE2Mjc4Ng==; proven_owner_popup=1; gdpr=0; cycada=qoe+imELqqJpjikNGwnhdUsrQq+wD0CX1ce2Zi0+Fbg=; _ym_uid=1606486172951512071; _ym_d=1606833021; X-Vertis-DC=myt; _ym_isad=2',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru',
    'Referer': 'https://auto.ru/moskva/cars/vendor-foreign/all',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0',
    'x-client-app-version': '202011.26.150101',
    'x-client-date': '1606833028238',
    'x-csrf-token': '3e2b10b5c34f56ca20cceb9967b1ba5adae281a7cee3f71a',
    'x-page-request-id': '334132efb1fa23b799d9a16d5db8b726',
    'x-requested-with': 'fetch'
}

In [244]:
url = "https://auto.ru/-/ajax/desktop/listing/"

In [245]:
# посмотрим какие столбцы нам нужны 
test = pd.read_csv('test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
test.sample(2)

Кол-во столбцов теста: 32
Названия столбцов теста: ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
21387,внедорожник 5 дв.,TOYOTA,https://auto.ru/cars/used/sale/toyota/land_cru...,чёрный,"{""id"":""21131790"",""name"":""Престиж"",""available_o...",Комплектация Престиж. Продаётся в связи с поку...,4.0 LTR,249 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,9000,2017,"{""code"":""LAND_CRUISER_PRADO"",""name"":""Land Crui...",LAND_CRUISER_PRADO,4.0 AT (249 л.с.) 4WD,5,1603295338,RUB,2018,1091237202,"{""id"":""21090309"",""displacement"":3956,""engine_t...",ALLROAD_5_DOORS AUTOMATIC 4.0,автоматическая,JAPANESE,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
6919,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s90/10963...,пурпурный,NaN,Автомобиль в нормальном рабочем состоянии. Не ...,2.9 LTR,180 N12,"{""airbag-driver"":true,""windcleaner-heat"":true,...",бензин,https://autoru.naydex.net/ESooa9149/468b9eoMem...,400000,1996,"{""code"":""S90"",""name"":""S90"",""ru_name"":""S90"",""mo...",S90,2.9 MT (180 л.с.),4,1603237831,RUB,1998,1096318170,"{""id"":""6016484"",""displacement"":2922,""engine_ty...",SEDAN MECHANICAL 2.9,механическая,EUROPEAN,2 владельца,1 год и 7 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен


In [246]:
test.equipment_dict[1]

'{"cruise-control":true,"asr":true,"esp":true,"airbag-driver":true,"isofix":true,"usb":true,"light-sensor":true,"airbag-passenger":true,"computer":true,"wheel-power":true,"alarm":true,"lock":true,"fabric-seats":true,"electro-mirrors":true,"drl":true,"electro-window-back":true,"alloy-wheel-disks":true,"16-inch-wheels":true,"ptf":true,"rain-sensor":true,"airbag-side":true,"tyre-pressure":true,"mirrors-heat":true,"dark-interior":true,"driver-seat-updown":true,"wheel-heat":true,"abs":true,"front-seats-heat":true,"bluetooth":true,"wheel-configuration2":true,"immo":true,"windscreen-heat":true,"climate-control-2":true,"park-assist-r":true}'

In [247]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов


# но сначала установим главные параметры запроса
mark = 'BMW'  # марка
model = 'M6'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "1"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '0000CC', 'delivery_info': {}, 'description': 'Премиальная акустическая система объемного звука Bang & Olufsen с 16 динамиками\nАдаптивный круиз-контроль\nСигнализация схода с полосы движения\nПроекция на лобовое стекло\n\nАвтомобиль в идеальном внешнем и техническом состоянии.\nОбслуживание автомобиля исключительно у официального дилера, все ТО согласно регламенту.\nСЕМЬ регистрационных действий, ТРИ Владельца за все время эксплуатации.\n\n"Royal Motors Club"  предоставляет полный спектр услуг по продажам автомобилей с пробегом:\n- Выгодные условия кредитования, одобрение по двум документам.\n- Trade In – обмен любого Вашего авто на представленный в нашей компании это\n   быстро, выгодно, безопасно!\n-  Мы осуществили все необходимые проверки и даем гарантию юридической\n   чистоты всех наших автомобилей.\n - Оптимальные программы по авто-страхованию ( ОСАГО, КАСКО)\n\nАвтомобиль прошел комплексную диагностику и предпродажн

In [248]:
# строка для проверки содержания элемента запроса
elem['additional_info']['hidden']

False

In [249]:
def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [252]:
# создаем вспомагательный словарь, в который будем записывать необходимые данные из каждого объявления
dict_columns = {'bodyType':0, 'brand':0, 'car_url':0, 'color':0, 'complectation_dict':0,
       'description':0, 'engineDisplacement':0, 'enginePower':0, 'equipment_dict':0,
       'fuelType':0, 'image':0, 'mileage':0, 'modelDate':0, 'model_info':0, 'model_name':0,
       'name':0, 'numberOfDoors':0, 'parsing_unixtime':0, 'priceCurrency':0,
       'productionDate':0, 'sell_id':0, 'super_gen':0, 'vehicleConfiguration':0,
       'vehicleTransmission':0, 'vendor':0, 'Владельцы':0, 'Владение':0, 'ПТС':0,
       'Привод':0, 'Руль':0, 'Состояние':0, 'Таможня':0, 'price':0}


# создаем функцию для обработки 23 необходимых столбцов из объявления в словарь
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['name']
    except: d_dict['brand'] = None

    d_dict['car_url'] = None
        
    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None
        
    try: d_dict['complectation_dict'] = d_elem['vehicle_info']['complectation']
    except: d_dict['complectation_dict'] = None
   
    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None    
        
    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['displacement']
    except: d_dict['engineDisplacement'] = None
    
    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None
    
    try: d_dict['equipment_dict'] = elem['vehicle_info']['equipment']
    except: d_dict['equipment_dict'] = None

    try: d_dict['fuelType'] = d_elem['vehicle_info']['tech_param']['engine_type']
    except: d_dict['fuelType'] = None    
        
    d_dict['image'] = None
        
        
    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None       
    
    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None
            
    try: d_dict['model_info'] = d_elem['vehicle_info']['model_info']
    except: d_dict['model_info'] = None

    try: d_dict['model_name'] =  d_elem['vehicle_info']['model_info']['code']
    except: d_dict['model_name'] = None
        
    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None
    
    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None
        
    try: d_dict['parsing_unixtime'] = d_elem['additional_info']['fresh_date']
    except: d_dict['parsing_unixtime'] = None

    try: d_dict['priceCurrency'] = d_elem['price_info']['currency']
    except: d_dict['priceCurrency'] = None
    
    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    
    try: d_dict['sell_id'] = d_elem['price_history']['saleid']
    except: d_dict['sell_id'] = None
        
    try: d_dict['super_gen'] = d_elem['vehicle_info']['super_gen'] 
    except: d_dict['super_gen'] = None
    
    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']
    except: d_dict['vehicleConfiguration'] = None
    
    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None
        
    try: d_dict['vendor'] = d_elem['vehicle_info']['vendor']
    except: d_dict['vendor'] = None

    
    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Привод'] = d_elem['vehicle_info']['tech_param']['gear_type']
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None
    
     # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    return

# тест функции 
from_elem_to_dict(elem, dict_columns)

In [253]:
# проверяем формат тестового запроса с сайта и теста с кагл
display(pd.DataFrame(dict_columns).sample(2))
test[test['enginePower']=='560 N12'].sample(2)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
hcc,Купе,BMW,None,0000CC,NaN,Премиальная акустическая система объемного зву...,4395,560,True,GASOLINE,None,67000,2012,NaN,M6,4.4 AMT (560 л.с.),2,1606118553000,RUR,2012,None,NaN,NaN,ROBOT,EUROPEAN,3,None,DUPLICATE,REAR_DRIVE,LEFT,None,True,2500000
name,Купе,BMW,None,0000CC,M6,Премиальная акустическая система объемного зву...,4395,560,NaN,GASOLINE,None,67000,2012,M6,M6,4.4 AMT (560 л.с.),2,1606118553000,RUR,2012,None,III (F06/F13/F12),NaN,ROBOT,EUROPEAN,3,None,DUPLICATE,REAR_DRIVE,LEFT,None,True,2500000


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
8457,купе,BMW,https://auto.ru/cars/used/sale/bmw/m6/10998759...,красный,"{""id"":""7928504"",""name"":""M6"",""available_options...","В продаже автомобиль от компании АО ABTODOM, о...",4.4 LTR,560 N12,"{""cruise-control"":true,""esp"":true,""usb"":true,""...",бензин,https://autoru.naydex.net/lICN99c69/9bc7d3Rr7k...,42978,2012,"{""code"":""M6"",""name"":""M6"",""ru_name"":""М6"",""morph...",M6,4.4 AMT (560 л.с.),2,1603111221,RUB,2013,1099875908,"{""id"":""7900064"",""displacement"":4395,""engine_ty...",COUPE ROBOT 4.4,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен
2385,лифтбек,AUDI,https://auto.ru/cars/used/sale/audi/rs7/110038...,серый,NaN,Audi Rs7 VSperformance Stage3\nНовый усиленный...,4.0 LTR,560 N12,NaN,бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,39900,2013,"{""code"":""RS7"",""name"":""RS 7"",""ru_name"":""РС7"",""m...",RS7,4.0 AT (560 л.с.) 4WD,5,1603121542,RUB,2013,1100381316,"{""id"":""20045126"",""displacement"":3993,""engine_t...",LIFTBACK AUTOMATIC 4.0,автоматическая,EUROPEAN,3 или более,2 года и 4 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [254]:
from time import perf_counter 
import time

### 4. Парсинг

In [255]:
auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = perf_counter()

for mark in list_of_relevant_marks:
    
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        start_time = perf_counter()
        for num_page in range(1, 200):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    from_elem_to_dict(elem, dict_columns)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.perf_counter() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(perf_counter() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    #print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.')

    old_iter_of_ads=iter_of_ads

print (f'Всего спарсили := {iter_of_ads} строк')

Всего спарсили := 73873 строк


In [256]:
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('auto_ru_v3_30_11_2020.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')

кол-во строк датасета:= 73873


In [258]:
df_auto_ru.sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
19485,Внедорожник 5 дв.,Hyundai,None,FAFBFB,{'id': '0'},None,1999,149,"{'electro-window-back': True, 'ptf': True, 'el...",GASOLINE,None,50522,2016,"{'code': 'CRETA', 'name': 'Creta', 'ru_name': ...",CRETA,2.0 AT (149 л.с.),5,1606237049000,RUR,2017,None,"{'id': '20773960', 'name': 'I', 'ru_name': '1'...","{'id': '20773995', 'body_type': 'ALLROAD_5_DOO...",AUTOMATIC,KOREAN,1,None,ORIGINAL,FORWARD_CONTROL,LEFT,None,True,1230000
11680,Седан,Chevrolet,None,97948F,"{'id': '2423094', 'name': 'SE', 'available_opt...","Автомобиль в хорошем состоянии,менялось все во...",1498,86,"{'airbag-driver': True, 'seat-transformation':...",GASOLINE,None,174000,2002,"{'code': 'LANOS', 'name': 'Lanos', 'ru_name': ...",LANOS,1.5 MT (86 л.с.),4,1598995020000,RUR,2006,None,"{'id': '2417263', 'name': 'I', 'year_from': 20...","{'id': '2417282', 'body_type': 'SEDAN', 'doors...",MECHANICAL,AMERICAN,3,None,ORIGINAL,FORWARD_CONTROL,LEFT,None,True,180000
5602,Седан,BMW,None,040001,"{'id': '21385509', 'name': '520d Executive', '...","Авто без аварий , резина зима / лето. Комплект...",1995,190,"{'cruise-control': True, 'asr': True, 'esp': T...",DIESEL,None,70000,2016,"{'code': '5ER', 'name': '5 серии', 'ru_name': ...",5ER,520d 2.0d AT (190 л.с.),4,1552456999000,RUR,2017,None,"{'id': '20856169', 'name': 'VII (G30/G31)', 'r...","{'id': '20856206', 'body_type': 'SEDAN', 'door...",AUTOMATIC,EUROPEAN,1,"{'year': 2017, 'month': 8}",ORIGINAL,REAR_DRIVE,LEFT,None,True,2480000
61119,Внедорожник 5 дв.,Great Wall,None,CACECB,{'id': '0'},Продается в связи с новым проектом - внедорожн...,2237,105,"{'steel-wheels': True, 'esp': True, 'ptf': Tru...",GASOLINE,None,115000,2001,"{'code': 'SAFE', 'name': 'Safe', 'ru_name': 'С...",SAFE,2.2 MT (105 л.с.) 4WD,5,1530889617000,RUR,2007,None,"{'id': '2475450', 'year_from': 2001, 'year_to'...","{'id': '2475466', 'body_type': 'ALLROAD_5_DOOR...",MECHANICAL,CHINESE,1,"{'year': 2008, 'month': 1}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,None,True,349999
41972,Компактвэн,Opel,None,FAFBFB,{'id': '0'},КЛЮЧАВТО автомобили с пробегом — направление ...,1796,140,"{'electro-window-back': True, 'leather-gear-st...",GASOLINE,None,152000,2008,"{'code': 'ZAFIRA', 'name': 'Zafira', 'ru_name'...",ZAFIRA,1.8 MT (140 л.с.),5,1605312206000,RUR,2011,None,"{'id': '2308407', 'name': 'B Рестайлинг', 'yea...","{'id': '2308408', 'body_type': 'COMPACTVAN', '...",MECHANICAL,EUROPEAN,2,None,ORIGINAL,FORWARD_CONTROL,LEFT,None,True,479000


In [220]:
# визуальный контроль
df_auto_ru['Владение'].value_counts()


Series([], Name: Владение, dtype: int64)